# Validation of two-dimensional video-based inference of finger

## OpenPose example with Python integration and coordinates output
This is an example to evaluate the output keypoints for image from the any dataset.

## Resources
- OpenPose documentation: https://cmu-perceptual-computing-lab.github.io/openpose/html/index.html
- Master Thesis that uses the Python API: https://csusm-dspace.calstate.edu/bitstream/handle/10211.3/217145/BhosaleRadhika_Summer2020.pdf?sequence=3

## Download and installation


In [ ]:
import sys
import cv2
import os
from sys import platform
import argparse
import time

In [ ]:
#@title
#Install CMake
!wget -q https://cmake.org/files/v3.18/cmake-3.18.2-Linux-x86_64.tar.gz

In [ ]:
#@title
#Unzip the tar file
!tar xfz cmake-3.18.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

In [ ]:
#@title
#Clone the openpose github repo
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
!git clone -q --depth 1 $git_repo_url

In [ ]:
#@title
#Commands to fix a directory in caffe
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt

In [ ]:
#@title
#Install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

In [ ]:
#@title
#Before this cell, change the CMakeLists.txt file: on line 317 and 798, switch option build Python to ON
#Build openpose
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [ ]:
#@title
#Before this cell, upload the 02_whole_body py file in build/examples/tutorial_api_python
#Upload figure to analyze and create new directories
!mkdir coordinates-out
!mkdir images-out
!mkdir images-in 

In [ ]:
#@title
import os
os.chdir("/content/openpose/build/examples/tutorial_api_python")

## Import image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
%matplotlib inline
import numpy as np
n="01"
fig = plt.figure(figsize=(15,15))
testim = mpimg.imread("../../../../images-in/"+n+".jpg")

## Run the model, output coordinates and figure with keypoints

In [ ]:
#@title
!python3 02_whole_body_from_image.py --hand --image_path "../../../../images-in/01.jpg"
#testim_out = mpimg.imread("hand2_w_points.png")
right_hand_2d = np.loadtxt("right_hand_keypoints.txt")
left_hand_2d = np.loadtxt("left_hand_keypoints.txt")
np.savetxt("../../../../coordinates-out/rh"+n+".txt",right_hand_2d)
np.savetxt("../../../../coordinates-out/lh"+n+".txt",left_hand_2d)

In [ ]:
fig = plt.figure(figsize=(15,15))
plt.scatter(right_hand_2d.T[0],right_hand_2d.T[1])
plt.scatter(left_hand_2d.T[0],left_hand_2d.T[1])
plt.savefig("../../../../images-out/s"+n+"o.jpg")

## Number the keypoints for reference

In [ ]:
pr = ['1r','2r','3r','4r','5r','6r','7r','8r','9r','10r','11r','12r','13r','14r','15r','16r','17r','18r','19r','20r','21r']
pl = ['1l','2l','3l','4l','5l','6l','7l','8l','9l','10l','11l','12l','13l','14l','15l','16l','17l','18l','19l','20l','21l']
plt.scatter(right_hand_2d.T[0], right_hand_2d.T[1],c=blue_a)
plt.scatter(left_hand_2d.T[0], left_hand_2d.T[1],c=blue_a)

for i,s in enumerate(pr):

  xr = right_hand_2d.T[0][i]
  yr = right_hand_2d.T[1][i]
  plt.annotate(s, # this is the text
                (xr,yr), # this is the point to label
                textcoords="offset points", # how to position the text
                xytext=(0,10), # distance from text to points (x,y)
                ha='center') # horizontal alignment can be left, right or center

for i,s in enumerate(pl):

  xl = left_hand_2d.T[0][i]
  yl = left_hand_2d.T[1][i]
  plt.annotate(s, # this is the text
                (xl,yl), # this is the point to label
                textcoords="offset points", # how to position the text
                xytext=(0,10), # distance from text to points (x,y)
                ha='center') # horizontal alignment can be left, right or center

## Example measurement: thumb abduction (TAB) angle

Measurement location based on:
Carpinella I, Mazzoleni P, Rabuffetti M, Thorsen R, Ferrarin M. Experimental protocol for the kinematic analysis of the hand: definition and repeatability. Gait & Posture. 2006 Jun 1;23(4):445-54.

In [ ]:
#Coordinates to use for measuring TAB angle:
for n in [1,2,5]:
  print(right_hand_2d[n])

In [ ]:
lm = ['2r','3r','6r']
right_hand_2d_m = [[right_hand_2d[1][0], right_hand_2d[2][0], 
                    right_hand_2d[5][0]], 
                   [right_hand_2d[1][1], right_hand_2d[2][1], 
                     right_hand_2d[5][1]]]
plt.scatter(right_hand_2d_m[0], right_hand_2d_m[1],c=blue_a)

for i,s in enumerate(lm):

  x = right_hand_2d_m[0][i]
  y = right_hand_2d_m[1][i]
  plt.annotate(s, # this is the text
                (x,y), # this is the point to label
                textcoords="offset points", # how to position the text
                xytext=(0,10), # distance from text to points (x,y)
                ha='center') # horizontal alignment can be left, right or center

In [ ]:
import math
def measure_angle_given_coords(x1y1, x2y2, x3y3):
  """Function that measures the angle given 3 coordinates
    Args:
      x1y1 (list): coordinates of the vertex where the angle is measured.
      x2y2 (list): coordinates of the second vertex.
      x3y3 (list): coordinates of the third vertex.

    Returns:
      alpha: the angle measured in degrees.
  """
  x1,y1 = x1y1
  x2,y2 = x2y2
  x3,y3 = x3y3
  ab = [x2-x1, y2-y1]
  ac = [x3-x1, y3-y1]
  dot_product = np.dot(ab,ac)
  abs_ab = math.sqrt(ab[0]**2+ab[1]**2)
  abs_ac = math.sqrt(ac[0]**2+ac[1]**2)
  cos_alpha = dot_product/(abs_ab*abs_ac)
  alpha = math.degrees(math.acos(cos_alpha))
  return alpha

In [ ]:
#Print angles in output 
alpha_ml = measure_angle_given_coords(right_hand_2d[1], right_hand_2d[2], right_hand_2d[5])
print(alpha_ml)